# Telegram Chatbot
Dieses Notbook enthält den Code, um über einen Telegram Chatbot mit ChatGPT zu schreiben.

Das Konzept dieses Programs ist unkompliziert. 
1. Ein Nutzer schreibt im Telegram Web eine Nachricht.
2. Das Programm prüft, ob es neue Nachricht (aller 5 Sekunde) im Chat gibt.
3. Falls es eine neue Nachricht findet, leitet es sie an ChatGPT.
4. Sobald ChatGPT antwortet, leitet das Programm die Antwort zurück an Telegram
5. Die Vorgänge 1.-4. wiederholen sich, bis der Nutzer das Programm abbricht.

Die Herausforderungen hier sind die Einstellung des ChatGPT und die Struktur der Telegram-Nachrichten zu verstehen, um die richtige Information zu extrahieren. Außerdem müssen wir das Programm anweisen, was zu tun, wenn es nicht funktioniert, sodass das Programm nicht fehlschlagen. Deshalb ist das Code immer so lang und sieht kompliziert.

*Das Code wurde inspiriert von https://dev.to/leighola/creating-a-telegram-chatbot-with-chatgpt-a-step-by-step-guide-42gg*

### Installieren notwendiger Bibliotheken

In [ ]:
!pip install python-dotenv
!pip install playwright
!pip install openai

!playwright install 

### Importieren der Bibliotheken
In der Programmiersprache Python müssen wir zu Beginn eines Programms die Bibliotheken importieren, die für das Programm benötigt werden. Für diesen Chatbot benötigen wir die folgenden Bibliotheken:

* `time` : zum periodischen Abfragen der Nachrichten
* `re` : zum Überprüfen der Nachrichten
* `os` : zum Abfragen der Umgebungsvariablen (OpenAI und Telegram Token)
* `requests` : erlaubt Web-Anfragen
* `openai`: entält das Interface zu ChatGPT
* `asyncio` : für asynchrone Webanfragen

Man verwendet den Befehl `import`, um diese Bibliotheken zu importieren.

Manchmal benötigen wir nur eine Funktion der Bibliothek. Anstatt die gesamte Bibliothek zu importieren, können wir auch nur die Funktion importieren. In diesem Fall schreibenn wir z.B.

* `from dotenv import load_dotenv` : erlaubt das definieren von Umgebungsvariablen
* `from playwright.async_api import async_playwright` : für asynchrone Webanfragen

**Aufgabe: Bitte die fehlende Bibliotheken importieren**

In [1]:
from dotenv import load_dotenv # erlaubt das definieren von Umgebungsvariablen
import time # zum periodischen Abfragen der Nachrichten

## Bitte die fehlende Bibliotheken [re, os, requests, openai] importieren
import re # zum Überprüfen der Nachrichten
import os # zum Abfragen der Umgebungsvariablen (OpenAI und Telegram Token)
import requests # erlaubt Web-Anfragen
import openai # entält das Interface zu ChatGPT

import asyncio # für asynchrone Webanfragen
from playwright.async_api import async_playwright # für asynchrone Webanfragen, um mit ChatGPT zu interagieren

### Definieren der Umgebungsvariable
Zunächst müssen wir unsere Umgebungsvariablen einrichten. Hier nutzen wir 3 Umgebungsvariablen:
1. (Nötig) `OPENAI_API_KEY`, die ein "Schlüssel" für den ChatGPT Zugang speichert; und
2. (Nötig) `TELEGRAM_TOKEN`, die ein Telegram Token (wie ein Ausweis / eine Eintrittskarte) speichert; und
3. (Optional) `CHAT_ID`, die eine kommagetrennte Sammlung der Chat-IDs der Chats speichert.

Die Umgebungsvariable `OPENAI_API_KEY` speichert der OpenAI-API-Schlüssel. Dieser Schlüssel ist eine eindeutige Kennung, die für den Zugriff auf die OpenAI-API erforderlich ist, die uns den Zugang zu ChatGPT ohne Web-Browser ermöglicht. 

Die Umgebungsvariable `TELEGRAM_TOKEN` wird verwendet, um das Token für unseren Telegram-Bot zu speichern. Dies ist eine lange Zeichenfolge aus Buchstaben und Zahlen, die zur Authentifizierung unseres Bots bei der Telegram-API verwendet wird. Wir benötigen dieses Token, um Nachrichten von Telegram senden und empfangen zu können.

Die Umgebungsvariable `CHAT_ID` ist **optional** und wird verwendet, um eine kommagetrennte Sammlung der Chat-IDs der Chats zu speichern, die unseren Bot verwenden dürfen. Die Chat-ID ist eine eindeutige Kennung für jeden Chat auf Telegram, und wir können sie verwenden, um nur bestimmten Benutzern die Nutzung unseres Bots zu erlauben. Auf diese Weise können wir unseren Bot privat halten und nur unseren Freunden erlauben, ihn zu benutzen.

In [ ]:
# Bitte dein OpenAI API-Key, Telegram-Bot Token und (optional) Chat-IDs hier geben.
OPENAI_API_KEY = "<Generated OpenAI Key>" # Bitte dein OpenAI API Key hier geben.
TELEGRAM_TOKEN= "<Dein Telegram Token hier>"  ## z.B.123456:ABC-DEF1234ghIkl-zyx57W2v1u123ew11
CHAT_ID = "<Deine Chat-IDs hier>"  ## z.B. -246810121416, 0123456789

#### Hinweise über die Handlung der sensible Information wie API-Schlüssel und Token.

Wenn wir sensible Informationen (z.B. Passwörter) haben, die wir nicht explizit in unserem Code speichern wollen. Wir schreiben diese Informationen in eine .env-Datei. Diese Datei wird aber nicht an die Versionskontrolle übergeben oder mit anderen geteilt.

Durch den Aufruf von `load_dotenv()` können wir den Inhalt der .env-Datei in die Umgebungsvariablen des Systems laden, sodass unser Python-Skript auf sie zugreifen kann. Auf diese Weise ist es einfach, sensible Informationen aus unserem Code herauszuhalten und Code weiterzugeben, ohne sensible Informationen weiterzugeben.

Korrekterweise hätten wir einen Projektordner anlegen und eine .env-Datei in diesen Ordner schreiben müssen. Für diesen Workshop werden wir jedoch zur Vereinfachung unsere sensiblen Informationen direkt in unseren Code schreiben. **TU DIES NICHT IN DER PRAXIS!**

In [ ]:
## Erstellen eine .env-Datei im Stammverzeichnis unseres Projekts
with open(".env","w+") as file:
   
    ## Erstellen die Umgebungsvariablen TELEGRAM_TOKEN 
    file.write(f"TELEGRAM_TOKEN={TELEGRAM_TOKEN}\n")
    file.write(f"OPENAI_API_KEY={OPENAI_API_KEY}\n")

    ## Falls CHAT_ID angegeben ist, schreiben die Chat-IDs auch als eine Umgebungsvariable 
    if len(CHAT_ID)>0:
        file.write(f"CHAT_ID={CHAT_ID}\n") 

Wir verwenden die Funktion `load_dotenv`, um die Umgebungsvariablen aus einer Datei namens .env zu laden, die sich im selben Verzeichnis wie unser Skript befindet.

In [3]:
## Laden Umgebungsvariablen aus .env-Datei.
load_dotenv()

## Geben OpenAI Schlüssel ein.
openai.api_key = os.environ['OPENAI_API_KEY']

### Webanfragen und ChatGPT

Beim Programmieren ist eine Funktion eine Reihe von Anweisungen, die wir immer dann wiederverwenden können, wenn wir eine bestimmte Aufgabe in unserem Code ausführen müssen. 

In Python definieren wir eine Funktion mit einem Format
```
def function_name(Eingaben):
    Anweisungen oder Vorgehen

    return Ausgabe
```

Um die Funktion zu benutzen, schreiben wir
```
function_name(Eingaben)
```

Im folgenden Abschnitt definieren wir die benötigenden Funktionen.

#### Methoden, um mit ChatGPT zu kommunizieren
Wir brauchen 2 Funktionen, um mit ChatGPT zu kommunizieren.

1. `message_to_chatgpt(message)` gibt die Einstellungen an ChatGPT an und schickt Nachrichten an ChatGPT. Mit der Schlüsselwörter (*Keyword arguments*) `model`,`temperature`,`presence_penalty`,`frequency_penalty` könn wird die Sprachmodellversion (`model`) auswählen und die Charakteristiken der Antwort einstellen. Mit dem Schlüsselwort `messages` geben wir unsere Nachrichten an. Die Nachrichten müssen dieses Format haben:
```
messages = [{"role":"entweder user, assistant, system":"Prompt 1"},
            {"role":"entweder user, assistant, system":"Prompt 2"},
            ...
            {"role":"entweder user, assistant, system":"Letztes Prompt = Nachricht an ChatGPT"}]
```
Die Rolle `user`,`assistant`,`system` weist ChatGPT hin, von "wem" ist die Nachricht. Das Verhalten des ChatGPT kann nach dieser Rolle variieren. Wenn wir wollen, dass ChatGPT mit einer bestimmten Stil antworten, geben wir das Prompt aus der Rolle `system` ein. Zum Beispiel
```
messages = [{"role":"system":"Imagine you are a 5-year old girl. You response to everything like her."},
            ...]
```
Das Prompt `"Imagine you are a 5-year old girl. You response to everything like her."` weist ChatGPT an, dass es wie eine Fünfjährige antworten muss. ChatGPT wird nicht auf diese Anweisung keine Rückmeldung geben. Es antwortet nur auf die letzte Nachricht.

*Für detailierte Information über die mögliche Einstellungen von ChatGPT, siehe https://platform.openai.com/docs/api-reference/chat (in Englisch)*   

2. Die `send_and_receive(message, trial=1)` Funktion ruft die Funktion `message_to_chatgpt(message)` ab und prüf, ob ChatGPT antwortet. Sie versucht 3 Mal die Nachricht zu schicken, falls sie keine Antwort sofort bekommt. Ansonsten bricht die Aktion ab.

In [ ]:
def message_to_chatgpt(message):
    """ 
    Die Funktion die Einstellungen für ChatGPT geben. Dann schickt eine Nachricht (Prompt) an ChatGPT. 
    Die Funktion gibt die Antwort von ChatGPT basierend auf der Einstellung zurück.
    """
    ## Wenn wir die Antwort-Stil ändern möchten, ändern wir die Code-Reihe unten und gibt an, in wessen Stil
    ## ChatGPT antworten soll. 
    ## Hier sollte ChatGPT als ein Persönlichkeittest sein.
    ## Probiere: "Imagine you are a 5-year old girl. You response to everything like her."
    message_prompt = "Du bist ein Chatbot, der einen Persönlichkeitstest, um heraus zu finden welches Tier die Person ist. Die Person schreibt etwas über sich. Als Ergebnis gibst du ein Tier zurück, das auf der Persönlichkeit der Person basiert. Starte nach dieser Nachricht mit einer kurzen Erklärung. Erkäre, dass die Person kurz etwas über ihre Hobbys und Interessen erzählen soll und du basierend darauf bestimmst, welches Tier sie ist."
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  ## Alternativ: gpt-3.5-turbo-0301
        
        ## Hier kann man die Charakteristiken der Antwort einstellen
        temperature = 1,  ## zwischen 0 und 2. Ein höher Wert macht die Antwort beliebiger.
        presence_penalty = 0,  ## zwischen -2.0 und 2.0. Positive Werte macht ChatGPT über neue Themen sprechen.
        frequency_penalty = 0, ## zwischen -2.0 und 2.0. Positive Werte senkt ChatGPTs Tendenz, sich mit ähnliche Sätze zu wiederholen.

        messages=[
            {"role": "system", "content": message_prompt},  ## Stilangabe für ChatGPT
            {"role": "user", "content": message}] ## ChatGPT wird auf dieser Nachricht antworten.
    )  
    return completion.choices[0].message.content

Probiere, ob die Funktion funktioniert. Schreibe eine Nachricht an ChatGPT über diese Funktion.

In [ ]:
## Ruf die Funktion messeage_to_chatgpt() ab, um Deine Nachricht an ChatGPT zu senden.
message_to_chatgpt("Erkläre, was eine Funktion beim Programmieren ist, an 11-jährige.")

In [ ]:
async def send_and_receive(message, trial=1):
    response = message_to_chatgpt(message)
    print(f"Response from chatGPT: '{response}'")
    
    ## Regular expression (regex) anwenden, um zu prüfen, ob die Rückmeldung leer ist.
    if (not response or re.match(r"^[^a-zA-Z0-9]$", response)) and trial <= 3:
        print("No response from chatGPT, trying again")
        return await send_and_receive(message, trial=trial*1.5)
    elif trial > 3:
        print("No response from chatGPT, giving up")
        return "<ChatGPT is not responding.>"
    return response

Um die Funktion mit dem Schlüsselwort `async` abzurufen, müssen wir das Schlüsselwort `await` benutzen.

In [ ]:
## Testen, ob die Funktion funktioniert.
await send_and_receive("What does the Python function load_env() do?")

### Methoden, um mit Telegram zu kommunizieren
Die Hauptfunktion hier ist die Funktion `check_for_new_updates()`. Diese Funktion
1. sucht nach eine neue Nachricht an ChatBot im URL https://api.telegram.org/bot`TELEGRAM_TOKEN`,
2. extrahiert die Nachricht, Chat-ID, und Message-ID (um auf bestimmte Nachricht zu antworten),
3. prüft mit Funktion `check_chat_id(chat_id)`, ob Chat-ID erlaubt ist (falls `CHAT_ID` vorhanden),
4. schickt die Nachricht an ChatGPT,
5. und leitet die Antwort von ChatGPT an Telegram weiter (mit der Funktion `send_message_to_telegram(message, chat_id, message_id)`.)

Wenn wir die Nachrichten an unser Bot abfragen mit dem URL `https://api.telegram.org/bot`TELEGRAM_TOKEN`/getUpdates`, bekommen wir die Ergebnisse im Format:
```
{"ok":true,"result":[
    {"update_id":447340877,
    "message":{"message_id":5
                ,"from":{"id":##########,"is_bot":false,"first_name":"Vorname","last_name":"Nachname","language_code":"de"}
                ,"chat":{"id":##########,"first_name":"Vorname","last_name":"Nachname","type":"private"}
                ,"date":1682420667
                ,"text":"Hello world!"}}]}
```
Das Code in der Funktion `check_for_new_updates()` geht in diese Ergebnisse durch, um die Information in `update_id`, `message_id`, `chat[id]`, `text` zu extrahieren.


Woher wissen wir, welche Befehle, Funktionen, und Schlüsselwörter gibt es und wann benutzen wir welche? Wir können hier nachschlagen: https://core.telegram.org/bots/api

In [ ]:
## Definieren globale Variablen
last_update = 0  ## entspricht `update_id`, ein Bezeichner für die Aktualisierung. 
url = f"https://api.telegram.org/bot{os.environ['TELEGRAM_TOKEN']}"  ## URL für Webanfrage des Chatbots

def send_message_to_telegram(message, chat_id, message_id):
    global url  ## Die Funktion sollte die globale Variable "url" modifizieren.

    ## Beim senden der Nachrichten an Telegram müssen wir die Parameter für `chat_id` und `text` angeben.
    ## Der Parameter `reply_to_message` sagt zu Telegram, dass die Nachricht eine Antwort für eine bestimmte Nachricht ist.
    params = {"chat_id": chat_id, "reply_to_message_id": message_id, "text": message}
    
    ## Schicken eine Nachricht und speichern den Zustand der Nachricht (erfolgreich oder nicht)
    response = requests.get(url+"/sendMessage", params)
    if response.status_code == 200:  ## HTTP-Status Code 200 bedeutet die Anfrage (request) ist erfolgreich.
        print("Sent response to telegram successfully")
    else:
        print("Error sending response to telegram:", response.text)
    return None

def check_chat_id(chat_id):
    """
    Prüfen, ob Chat-IDs erlaubt sind.
    Die Funktion gibt den Wert "True" (= 1) zurück, falls Chat-IDs nicht gegeben oder ungültig sind. 
    """
    ## Prüfen, ob die Umgebungsvariable CHAT_ID existiert.
    try:
        chat_id_env = os.environ['CHAT_ID']
    except:
        return True  ## Falls CHAT_ID nicht existiert, gibt "True" zurürck
    if chat_id_env == "":
        return True  ## Falls kein CHAT_ID gegeben sind, gibt "True" zurück
    
    ## Sonst speichern Chat-IDs in einer Liste und gibt diese zurück
    ## Schreiben -246810121416, 0123456789 in [-246810121416, 0123456789], sodass Python sie lesen kann.
    chat_id_list = chat_id_env.split(',')  
    chat_id_list = [x.strip() for x in chat_id_list]  ## Entferne leere Zeichnen
    return str(chat_id) in chat_id_list


async def check_for_new_updates():
    """
    Suchen nach eine neue Nachricht im Chat.
    """
    global last_update  ## Die Funktion sollte die globale Variable "last_update" modifizieren.
    params = {"allowed_updates": ["message"]}  ## Wir wollen nur neue Nachrichten sehen. (Ansonsten bekommen wir andere Info dazu.)
    
    if last_update != 0:
        params['offset'] = last_update + 1  ## Abfragen nur die aktuelle Nachricht.

    print("Checking for updates")
    response = requests.get(url+"/getUpdates", params)
    if response.status_code == 200:  ## Falls die Webanfrage erfolgreich ist
        data = response.json()  ## Konvertieren die Ergebnisse der Abfrage ins Python-Dict Format für die Verarbeitung
        if data["ok"]:  ## Wenn data["ok"] == True: alles in Ordnung ist,
            if data["result"]:  ## Wenn die Ergebnis der Abfrage nicht leer ist, 
                ## Hier beginnt die Informationsextraktion.
                ## Die Ergebnisse der Abfrage ausgeben.
                for update in data["result"]:
                    print(update)
                    try:
                        ## Suchen nach eine Schlüsselwort "message" im Ergebnis. Falls nicht vorhanden, nehmen "edited_message"
                        key = 'message' if 'message' in update else 'edited_message'
                        last_update = update['update_id']  ## Die aktuelle Update-ID speichern.

                        try:
                            ## Abrufen Chat-ID und Message-ID
                            chat_id = update[key]["chat"]["id"]
                            message_id = update[key]["message_id"]
                            
                            ## Abrufen der aktuelle Nachrichten
                            message = update[key]["text"]
                        except:
                            ## Falls das Ergebnis keine gültige Nachricht enthält, informiert der Nutzer
                            ## und schauen die nächste Nachricht. Die Methode endet, wenn es keine weitere Nachrichten gibt.
                            print("This update is not a valid message or edited_message")
                            continue

                        ## Falls CHAT_ID angegeben werden und wir eine Nachricht aus anderen Chat abfragen,
                        ## wird diese Nachricht übersprungen. 
                        if not check_chat_id(chat_id):
                            print("Chat ID not allowed")
                            continue

                        ##  Senden die aktuelle Nachricht an ChatGPT und warten auf die Antwort
                        response = await send_and_receive(message)
                        
                        ## Senden die Antwort von ChatGPT an Telegram
                        send_message_to_telegram(response, chat_id, message_id)

                    except Exception as e:
                        print("Error processing update", update['update_id'], e)
                return data["result"][0]
            else:
                print("No new updates")
                return None
    ## Ausgeben Fehlermeldung und die Antwort
    print("Error getting updates:", response.text)

async def check_for_new_updates_periodically():
    """
    Suchen nach neue Nachrichten im Chat alle 5 Sekunden und schicken diese Nachricht an ChatGPT.
    """
    while True:
        await check_for_new_updates()
        time.sleep(5)

Die Funktion `start_browser()` startet unser Chatbot. Lass das Code unten laufen. Gehen zu https://web.telegram.org/ und in Telegram-Konto anhand QR Code einloggen. Jetzt können wir mit unserem Bot loschatten!

In [ ]:
async def start_browser():
    async with async_playwright() as playwright:
      await check_for_new_updates_periodically()

if __name__ == "__main__":
    asyncio.run(await start_browser())

----
<center> ENDE </center>